<a href="https://colab.research.google.com/github/everestso/Fall21Spring22/blob/main/c164s22ch3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tile Sliding Domain

In [ ]:
import random
import heapq

random.seed(13)

In [ ]:
StateDimension=3
# StateDimension=4
#InitialState = [1,2,3,4,5,6,0,7,8]
InitialState = "123456078"
#GoalState=[1,2,3,4,5,6,7,8,0]
GoalState = "123456780"
# GoalState = "123456789ABCDEF0"
Actions = lambda s: ['u', 'd', 'l', 'r']
Opposite=dict([('u','d'),('d','u'),('l','r'),('r','l'), (None, None)])

In [ ]:
def Result(state, action):
  i = state.index('0')
  newState = list(state)
  row,col=i//StateDimension, i % StateDimension
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return newState
  if action=='u':
    l,r = row*StateDimension+col, (row-1)*StateDimension+col
  elif action=='d':
    l,r = row*StateDimension+col, (row+1)*StateDimension+col
  elif action=='l':
    l,r = row*StateDimension+col, row*StateDimension+col-1
  elif action=='r' :
    l,r = row*StateDimension+col, row*StateDimension+col+1
  newState[l], newState[r] = newState[r], newState[l] 
  return ''.join(newState)

def PrintState(s):
  for i in range(0,len(s),StateDimension):
    print(s[i:i+StateDimension])

def LegalMove(state, action):
  i = state.index('0')
  row,col=i//StateDimension, i % StateDimension
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return False
  return True

In [ ]:
def SingleTileManhattanDistance(tile, left, right):
  leftIndex = left.index(tile)
  rightIndex = right.index(tile)
  return (abs(leftIndex//StateDimension-rightIndex//StateDimension) +
          abs(leftIndex%StateDimension-rightIndex%StateDimension))
  
def ManhattanDistance(left, right):
  distances = [SingleTileManhattanDistance(tile, left, right) 
     for tile in [str(c) for c in range(1, StateDimension**2)]]
  ### print ("Distances= ", distances)
  return sum(distances)

In [ ]:
def OutOfPlace(left, right):
  distances = [left[i]!=right[i] and right[i] != '0'
     for i in range(StateDimension**2)]
  return sum(distances)

In [ ]:
PrintState(InitialState)

123
456
078


In [ ]:
PrintState(GoalState)

123
456
780


In [ ]:
print("ManhattanDistance=  ", ManhattanDistance(InitialState, GoalState))
print("OutOfPlace= ", OutOfPlace(InitialState, GoalState))

ManhattanDistance=   2
OutOfPlace=  2


In [ ]:
PrintState(InitialState)
print()
state1 = Result(InitialState, 'u')
PrintState(state1)
print()
state1 = Result(state1, 'r')
PrintState(state1)

123
456
078

123
056
478

123
506
478


# Random Walk

In [ ]:
def RandomWalk(state, steps):
  actionSequence = []
  actionLast = None
  for i in range(steps):
    action = None
    while action==None:
      action = random.choice(Actions(state))
      action = action if (LegalMove(state, action) 
          and action!= Opposite[actionLast]) else None
    actionLast = action
    state = Result(state, action)
    actionSequence.append(action)
  return state, actionSequence

In [ ]:
state1, sol = RandomWalk(InitialState, 50)
PrintState(state1)
print (ManhattanDistance(state1, GoalState), sol)

state1, sol = RandomWalk(InitialState, 5)
PrintState(InitialState)
print (sol)
PrintState(state1)

840
627
135
18 ['u', 'u', 'r', 'd', 'l', 'd', 'r', 'r', 'u', 'l', 'l', 'd', 'r', 'r', 'u', 'u', 'l', 'd', 'r', 'd', 'l', 'u', 'u', 'l', 'd', 'd', 'r', 'u', 'r', 'u', 'l', 'l', 'd', 'r', 'r', 'd', 'l', 'u', 'r', 'd', 'l', 'l', 'u', 'r', 'r', 'u', 'l', 'd', 'r', 'u']
123
456
078
['r', 'u', 'u', 'l', 'd']
413
026
758


In [ ]:
def ApplyMoves(actions, state):
  for action in actions:
    state = Result(state, action)
  return state

In [ ]:
PrintState(InitialState)
print(['r','r'])
PrintState(ApplyMoves(['r','r'],InitialState))

123
456
078
['r', 'r']
123
456
780


In [ ]:
def ReverseMoves(actions):
  ret = [Opposite[a] for a in actions]
  ret.reverse()
  return ret

In [ ]:
state1, sol = RandomWalk(GoalState, 5)
PrintState(state1)
print (sol)
print(ReverseMoves(sol))
PrintState (ApplyMoves(ReverseMoves(sol), state1))

203
156
478
['l', 'l', 'u', 'u', 'r']
['l', 'd', 'd', 'r', 'r']
123
456
780


# Example 1

In [ ]:
Problems = [RandomWalk(GoalState, 5) for _ in range(10)]
for i, s in Problems:
  print ('"', i, '" , "', ''.join(map(str, ReverseMoves(s))), '",', 
         ManhattanDistance(i, GoalState), sep='')

"103526478" , "dldrr",5
"103526478" , "dldrr",5
"135420786" , "uldrd",5
"152043786" , "rurdd",5
"203156478" , "lddrr",5
"203156478" , "lddrr",5
"123576408" , "uldrr",5
"412053786" , "urrdd",5
"152430786" , "lurdd",5
"103526478" , "dldrr",5


In [ ]:
NewState = ApplyMoves("dldrr", "103526478")
print (NewState)
PrintState("123456780")
print()
PrintState("103526478")
print(OutOfPlace("103526478", "123456780"))
MD=[(1,0), (2, 1), (3, 0), (4, 1), (5, 1), (6, 0), (7, 1), (8, 1)]
print(ManhattanDistance("103526478", "123456780"))

123456780
123
456
780

103
526
478
5
5


In [ ]:
InitialState = "412053786"
GoalState = "123456780"
print ("ManhattanDistance=", ManhattanDistance(InitialState, GoalState))
print ("Out of Place= ", OutOfPlace(InitialState, GoalState))

ManhattanDistance= 5
Out of Place=  5


# Example 2

In [ ]:
Problems = [RandomWalk(GoalState, 100) for _ in range(20)]
for i, s in Problems:
  print ('"', i, '", ', ''.join(map(str, ReverseMoves(s))), '",', 
         OutOfPlace(i, GoalState), " ",  ManhattanDistance(i, GoalState), sep='')

"567234081", uurrddluurddllurrullddrurdluulddruulddrrululdrurdlurddlluurrdllurrddllururdlurdluldrdlurdlurrdllurdr",7 18
"820453761", llddrrullurrdlldrruldlurdluurrdlurddluurddluuldrrdllurdlurulddruuldrulddrruuldldrruulldrdrululddrurd",4 10
"067312845", ddrruuldrulddlurdlurdruullddrurulddrulldrruldruldruldlurruldldruuldrdruulddluurdlurrdllurrddlluurrdd",8 18
"281607543", uldrrdllurruldldruulddruurdldlurdruuldlurdlurdrdllururdlldrruullddruldrrulldruldrruuldrdlluurddlurrd",8 16
"783245610", uuldrulldrdluurdrulldrdlurrdlluurrddllurrdllurdlurdruullddruulddruuldrrdlluurdldrruuldrulldrrullddrr",7 14
"348107625", ulddrruulldrrdllurdruulldrruldrdlurdlluurrdlldruuldrrdluuldrrdlluruldrdrulldrrulldruurdldruldlururdd",8 18
"358174026", urdruullddrruullddrurdllurdlurulddrruullddrulurdldrurdlluurdrulldrurdlldruurdldluurrdlulddrrullurddr",8 14
"624508173", rulldrrdlluurrdldruldlururddlulurrdlldruruldrdllurruldlurdruldrdluldrruuldrulddluurddlurrulddluurrdd",7 14
"364851027", ruurdlurdldlurrdllurdruulld

In [ ]:
InitialState = "281607543"
GoalState = "123456780"
print ("ManhattanDistance=", ManhattanDistance(InitialState, GoalState))
print ("Out of Place= ", OutOfPlace(InitialState, GoalState))

ManhattanDistance= 16
Out of Place=  8


In [ ]:
InitialState = "076581324"
GoalState = "123456780"
print ("ManhattanDistance=", ManhattanDistance(InitialState, GoalState))
print ("Out of Place= ", OutOfPlace(InitialState, GoalState))
PrintState(InitialState)

ManhattanDistance= 18
Out of Place=  8
076
581
324


In [ ]:
sol = "drdlurrullddruruldlurrddlluurrddluldrruulddruullddrruullddrurdlulurrddluurdlulddrulurdldrurdluuldrdr"
print (len(sol))
print(ApplyMoves(sol, InitialState))

100
123456780


# Simple 15 Puzzle Test

In [ ]:
GoalState = "123456789ABCDEF0"
s15a = Result(GoalState, "l")
s15b = Result(GoalState, "u")

PrintState(GoalState)
print()
PrintState(s15a)
print('')
PrintState(s15b)

1234
5678
9ABC
DEF0

1234
5678
9ABC
DE0F

1234
5678
9AB0
DEFC
